# Gradient Boosting Classifier

#### Author: Yifan Wang
#### Theory Support:  Dr. Huang. Shangwen

Helpful Resources:
    
     https://en.wikipedia.org/wiki/Gradient_boosting
    
     https://www.quora.com/Why-does-GBM-use-regression-on-pseudo-residuals
     
     http://www.ccs.neu.edu/home/vip/teach/MLcourse/4_boosting/slides/gradient_boosting.pdf
     


### Key Ideas of Gradient Boosting:

    1. fit a weak learner
    2. identify and train a model on the pseudo residual and the gradient
    3. update the weak learner using the gradient of pseudo residual 
    (repeat step 2 and 3 until converge)
    
    

Sounds easy right? but it can be tricky to implement that:



In [1]:
from sklearn.datasets import load_breast_cancer
import numpy as np
from collections import Counter

#### Base Learner (will be used to learn the residuals)

In [2]:
class CART_regressor():
    'Implementation of CART(Classification and Regression Tree) Decision Tree in Python, majorly in NumPy'
    def __init__(self,least_children_num,verbose=True):
        self.least_children_num = least_children_num
        self.verbose = verbose
        
    
    def fit(self,tmp_x,tmp_y):
        def fit_tree(tmp_x,tmp_y):
        #     Exit Condition 0:
            # Exit Condition 1:
            if \
            len(tmp_y) < self.least_children_num or len(np.unique(tmp_y))==1:

                if self.verbose:
                    print('exit condition:')
                    print('tmp_y:')
                    print(tmp_y)

                mode_val = self.mean(tmp_y.flatten().tolist())
                return([np.nan, mode_val, np.nan, np.nan]) # Leaf Node: format [feat,splitval,]

            # Otherwise Split:
            if self.verbose:
                print("start....subset Y len {}".format(len(tmp_y)))


            split_row,split_col = self.decide_split_data(tmp_x,tmp_y)

            if not split_row and not split_col:
#                 print('no better split...return mean')
                mode_val = self.mean(tmp_y.flatten().tolist())
                return([np.nan, mode_val, np.nan, np.nan])

            if self.verbose:
                print("split on:")
                print(split_row,split_col)


            split_vec = tmp_x[:,split_col]
            split_val = tmp_x[split_row,split_col]

            # Recursively Split to left and right branches:
            left_ind = np.where(split_vec<split_val)[0].tolist()
            right_ind = np.where(split_vec>=split_val)[0].tolist()
            left_dat,left_y = tmp_x[left_ind,:],tmp_y[left_ind,]
            right_dat,right_y = tmp_x[right_ind,:],tmp_y[right_ind,]

            left_tree = fit_tree(left_dat,left_y)
            right_tree = fit_tree(right_dat,right_y)

            if isinstance(left_tree, list): # If list, tree len 1
                len_l_tree = 1
            else:
                len_l_tree = left_tree.shape[0] # If array, tree len >1

            root = [split_col,split_val,1,len_l_tree+1] # Format [split_col, split_val, left_tree_relative_idx, right_tree_relative_idx]
            return(np.vstack([root,left_tree,right_tree]))
        
        tree = fit_tree(tmp_x,tmp_y)
        self.tree = tree

    

    def decide_split_data(self,x,y):
        'Given subset of X,Y, search for the best splitting node based on: MSE reduction'
        def _MSE(tmp_y):
            'Key Metrics of building a decision tree. Specifically MSE'
            output = 0
            mean_val = np.mean(tmp_y)
            
            for i in range(len(tmp_y)):
                
                tmp  = (tmp_y[i] - mean_val)**2
                output += tmp
            output /= len(tmp_y)
            return output

        
        #---
        m,n = x.shape
        best_red = 0
        split_row, split_col = None,None

        previous_mse = _MSE(y)
        for col in range(n):
            tmp_vec = x[:,col].ravel()

            for row in range(m):
                val = tmp_vec[row]
                # >= & < is my convention here:
                if val!=np.max(tmp_vec) and val!= np.min(tmp_vec):
                    left_b = np.where(tmp_vec<val)[0].tolist()
                    right_b = np.where(tmp_vec>=val)[0].tolist()

                    new_mse = \
                    (len(y[left_b])/len(y))*_MSE(y[left_b]) + \
                    (len(y[right_b])/len(y))*_MSE(y[right_b])


    #                 print('new entropy: %f'%new_ent)
                    mse_red = previous_mse - new_mse

                    if mse_red > best_red:
                        split_row, split_col = row,col
                        best_red = mse_red
#                         if self.verbose:
#                             print('better red:{}'.format(mse_red))
#                             print()

        return split_row, split_col
                
                

    def mean(self, x_list):
        'calculate the mean'
        return np.mean(x_list)

    
    

    
    
    

    
    
    
    def predict(self, tmp_test_array):
        'Wrap-up fun for prediction'
        def query(tree,tmp_test_array):
            'Test for single example'
            assert len(tmp_test_array.shape) == 2, "Make sure your test data is 2d array"
        #     print(tree)
            if isinstance(tree,list):
                start_node = tree # only the 1 row in data
            else:
                start_node = tree[0,:] # Iteratively hit first row
                
    
            test_feat,test_val,left_tree_jump,right_tree_jump = start_node[0],start_node[1],start_node[2],start_node[3]

            # Exit Condition:
            if np.isnan(test_feat) and np.isnan(left_tree_jump) and np.isnan(right_tree_jump):
                pred = test_val;
                return pred 
            #Test:
            if tmp_test_array[0,int(test_feat)] < test_val:
                # If <, go left branch:
                jump_loc = left_tree_jump
                pred = query(tree[int(jump_loc):,],tmp_test_array)

            else:
                # If >=, go right branch:
                jump_loc = right_tree_jump
                pred = query(tree[int(jump_loc):,],tmp_test_array)

            return pred

        assert len(tmp_test_array.shape) == 2, "Make sure your test data is 2d array"
        result = []

        for i in range(tmp_test_array.shape[0]):
            inp = tmp_test_array[i,:].reshape(1,-1)
            result.append(query(self.tree,inp))
        return result

#### Load Data and CV Split:

In [3]:
X,y = load_breast_cancer(return_X_y=True)

idx = [i for i in range(len(y))]
np.random.seed(1028)
np.random.shuffle(idx)

In [4]:
val_ratio = 0.2
val_num = int(len(y)*val_ratio)

print("We will be using {} validation examples".format(val_num))

We will be using 113 validation examples


In [5]:
X_train,X_valid = X[val_num:], X[:val_num]
y_train,y_valid = y[val_num:], y[:val_num]

#### Gradient Boosting Classification Learner:

In [10]:
class Gradient_Boosting_Classifier():
    def __init__(self,LEAST_CHILDREN_NUM,LEARNING_RATE,N_TREE,VERBOSE=False):
        self.LEARNING_RATE = LEARNING_RATE
        self.LEAST_CHILDREN_NUM = LEAST_CHILDREN_NUM
        self.N_TREE = N_TREE
        self.trees = {}
        self.VERBOSE = VERBOSE
        
    def ohe(self,x):
        """
        One-Hot-Encoding a 1d Matrix
        """
        m = len(x)
        n = len(np.unique(x))
        output = np.zeros((m,n))
        for i in range(len(x)):
            output[i,int(x[i])] = 1
        return output
        
    def KL_divergence(self,arr_x,arr_y):
        """
        Calculate KL Divergence as our loss function
        """
        output = arr_x * np.log(arr_x / (arr_y+1)+1)
        return np.nansum(output, axis=0)

    def find_residual(self,y,pred):
        """
        find d_residual
        """
        n_class = pred.shape[1]
        d_loss = np.zeros(y.shape)
        for class_num in range(n_class):
            d_loss[:,class_num] = y[:,class_num] - pred[:,class_num]  # negative gradient
        return d_loss

        
    def fit(self,X,y):
        """
        sequentially train the boosting model
        """

        original_y = y.copy()
        # first ohe y:
        y = self.ohe(y)
        self.n_class = y.shape[1]
        # initialize:
        y_train_pred = np.zeros((len(y),self.n_class)) + (1.0/self.n_class)
        self.trees['initial_model'] = None # leave an interface if initial learner is a model
        
        
        
        
        
        
        for round_i in range(self.N_TREE):
            
            residual = self.find_residual(y_train_pred,y) # find residual
            print("Round-%d started"%round_i)
#             kl_div = self.KL_divergence(original_y,y_train_pred.argmax(axis=1))
#             print("KL Divergence {}".format(kl_div))
#             print()
            models = {}
            # fit on residual and update each class
            for class_i in range(self.n_class):
                model = CART_regressor(self.LEAST_CHILDREN_NUM,False)
                model.fit(X,residual[:,class_i])  
                models['model%d'%class_i] = model

                update_val = self.LEARNING_RATE*residual[:,class_i]
                y_train_pred[:,class_i] = y_train_pred[:,class_i] - update_val

            self.trees['tree%d'%round_i] = models # Save the trees for prediction
        
    def evaluate(self,X,y):
        """
        Evaluation,
        basically prediction, but requires label to evaluate
        """

        y_valid_pred = np.zeros((len(X),self.n_class)) + (1.0/self.n_class)
    
        for round_i in range(len(self.trees)-1): # exclude initial model

            models = self.trees['tree%d'%round_i]
            for class_i in range(self.n_class):
#                 print("round %d"%round_i)
#                 print('tree-%d'%round_i)
#                 print("class %d"%class_i)
                
                model = models['model%d'%class_i]
                temp_pred = model.predict(X)
#                 print(temp_pred)
                y_valid_pred[:,class_i] =  y_valid_pred[:,class_i] - self.LEARNING_RATE*np.array(temp_pred)
                
                del model
                del temp_pred
        
            
                
            temp_pred = y_valid_pred.argmax(axis=1)
            accuracy = sum(temp_pred==y)/len(y)
        print("Accuracy is %.4f"%accuracy)

        y_pred = y_valid_pred.argmax(axis=1)
        
        return y_pred



            


        

In [11]:
model = Gradient_Boosting_Classifier(
    LEARNING_RATE = 0.01,
    LEAST_CHILDREN_NUM = 10,
    N_TREE = 10, # 
    VERBOSE = True
)

In [12]:
model.fit(X_train,y_train)

Round-0 started
Round-1 started
Round-2 started
Round-3 started
Round-4 started
Round-5 started
Round-6 started
Round-7 started
Round-8 started
Round-9 started


#### cv prediction:

In [14]:
model.evaluate(X_train,y_train)

Accuracy is 0.9934


array([1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,

In [13]:
model.evaluate(X_valid,y_valid)

Accuracy is 0.8850


array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1])

### Out-of-sample accuracy 88.5%, slightly better than single CART classification tree